In [ ]:
# Importação das Bibliotecas
import cv2
import numpy as np
import os
import zipfile
from google.colab.patches import cv2_imshow
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
tf.__version__

'2.9.2'

In [ ]:
# !pip install tensorflow == 2.6.0

In [ ]:
# Extração dos pixels das imagens
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/redes-neurais/homer_bart_1.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
diretorio = '/content/homer_bart_1'
arquivos = [os.path.join(diretorio, f) for f in sorted(os.listdir(diretorio))]
print(arquivos)

['/content/homer_bart_1/.DS_Store', '/content/homer_bart_1/bart1.bmp', '/content/homer_bart_1/bart10.bmp', '/content/homer_bart_1/bart100.bmp', '/content/homer_bart_1/bart101.bmp', '/content/homer_bart_1/bart102.bmp', '/content/homer_bart_1/bart103.bmp', '/content/homer_bart_1/bart104.bmp', '/content/homer_bart_1/bart105.bmp', '/content/homer_bart_1/bart106.bmp', '/content/homer_bart_1/bart108.bmp', '/content/homer_bart_1/bart109.bmp', '/content/homer_bart_1/bart11.bmp', '/content/homer_bart_1/bart110.bmp', '/content/homer_bart_1/bart111.bmp', '/content/homer_bart_1/bart112.bmp', '/content/homer_bart_1/bart113.bmp', '/content/homer_bart_1/bart114.bmp', '/content/homer_bart_1/bart115.bmp', '/content/homer_bart_1/bart116.bmp', '/content/homer_bart_1/bart117.bmp', '/content/homer_bart_1/bart118.bmp', '/content/homer_bart_1/bart119.bmp', '/content/homer_bart_1/bart12.bmp', '/content/homer_bart_1/bart120.bmp', '/content/homer_bart_1/bart121.bmp', '/content/homer_bart_1/bart122.bmp', '/conte

In [ ]:
type(arquivos)

list

In [ ]:
largura, altura = 128, 128

In [ ]:
128 * 128

16384

In [ ]:
imagens = []
classes = []


In [ ]:
imagem.shape

NameError: ignored

In [ ]:
for imagem_caminho in arquivos:
  # print(imagem)_caminho
  try:
    imagem = cv2.imread(imagem_caminho)
    (H, W) = imagem.shape[:2]
  except:
    continue

  imagem = cv2.resize(imagem, (largura, altura))
  imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
  cv2_imshow(imagem)

  imagem = imagem.ravel()
  #print(imagem.shape)

  imagens.append(imagem)
  nome_imagem = os.path.basename(os.path.normpath(imagem_caminho))
  #print(nome_imagem)
  if nome_imagem.startswith('b'):
    classe = 0
  else:
    classe = 1

  classes.append(classe)
  print(classe)

In [ ]:
imagens

In [ ]:
print(classes)

In [ ]:
imagens[100]

In [ ]:
imagens[100], classes[100]

In [ ]:
imagens[200], classes[200]

In [ ]:
type(imagens), type(classes)

In [ ]:
X = np.asarray(imagens)
y = np.asarray(classes)

In [ ]:
type(X), type(y)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
sns.countplot(y)

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
# Normalização dos dados
X[0]

In [ ]:
X[0].max(), X[0].min()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
X[0].max(), X[0].min()

In [ ]:
X[1]

In [ ]:
# Bases de treinamento e teste
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
X_treinamento

In [ ]:
X_treinamento.shape, y_treinamento

In [ ]:
X_teste.shape, y_teste.shape

In [ ]:
(16384 + 2)/2

In [ ]:
# Construção e treinamento da rede neural
# 16384 -> 8193 (neuronios)
network1 = tf.keras.models.Sequential()
network1.add(tf.keras.layers.Dense(input_shape = (16384,), units=8193, activation='relu'))
network1.add(tf.keras.layers.Dense(units = 8193, activation='relu'))
network1.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

In [ ]:
network1.summary()

In [ ]:
# https://keras.io/api/optimizers
# https://keras.io/api/losses
network1.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
historico = network1.fit(X_treinamento, y_treinamento, epochs=50)

In [ ]:
# Avaliação da rede neural
historico.history.keys()

In [ ]:
plt.plot(historico.history['loss'])

In [ ]:
plt.plot(historico.history['accuracy'])

In [ ]:
X_teste.shape

In [ ]:
previsoes = network1.predict(X_teste)
previsoes

In [ ]:
# 0 False Bart
# 1 True Hommer
previsoes = (previsoes > 0.5)
previsoes

In [ ]:
y_teste

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_teste, previsoes)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_teste, previsoes)
cm

In [ ]:
sns.heatmap(cm, annot=True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_teste, previsoes))

In [ ]:
# Salvar e Carregar a rede neural
model_json = network1.to_json()
with open('network1.json', 'w') as json_file:
  json_file.write(model_json)

In [ ]:
from keras.models import save_model
network1_saved = save_model(network1, 'weights1.hdf5')

In [ ]:
with open('network1.json') as json_file:
  json_saved_model = json_file.read()
json_saved_model

In [ ]:
network1_loaded = tf.keras.models.model_from_json(json_saved_model)
network1_loaded.load_weights('/content/weights1.hdf5')
network1_loaded.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
network1_loaded.summary()

In [ ]:
# Classificação de uma única imagem
X_teste[0], y_teste[0] # Bart

In [ ]:
X_teste[0].shape

In [ ]:
cv2_imshow(X_teste[0].reshape(128,128))

In [ ]:
imagem_teste = X_teste[34] # Estava 0
imagem_teste = scaler.inverse_transform(imagem_teste.reshape(1, -1))

In [ ]:
imagem_teste

In [ ]:
cv2_imshow(imagem_teste.reshape(128,128))

In [ ]:
network1_loaded.predict(imagem_teste)[0][0]

In [ ]:
if network1_loaded.predict(imagem_teste)[0][0] < 0.5:
  print('Bart')
else:
  print('Homer')